In [8]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px

# Cargar datos
file_path = "modelo_con_predicciones.csv"  # Asegúrate de tener el archivo en la ruta correcta
df = pd.read_csv(file_path)

# Crear categorías de precio
bins = [df['y_actual'].min(), df['y_actual'].quantile(0.33), df['y_actual'].quantile(0.66), df['y_actual'].max()]
labels = ['Bajo', 'Medio', 'Alto']
df['price_category'] = pd.cut(df['y_actual'], bins=bins, labels=labels)

# Inicializar la aplicación Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout del dashboard
app.layout = dbc.Container([
    html.H1("Dashboard de Análisis de Alquileres"),
    
    dbc.Row([
        dbc.Col([
            html.Label("Seleccionar Ciudad:"),
            dcc.Dropdown(
                id='city-dropdown',
                options=[{'label': city, 'value': city} for city in df.filter(like='cityname_').columns],
                multi=True,
                placeholder="Seleccione ciudades"
            )
        ], width=6),
        dbc.Col([
            html.Label("Seleccionar Estado:"),
            dcc.Dropdown(
                id='state-dropdown',
                options=[{'label': state, 'value': state} for state in df.filter(like='state_').columns],
                multi=True,
                placeholder="Seleccione estados"
            )
        ], width=6)
    ]),
    
    html.Hr(),
    
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='scatter-price-sqft')
        ], width=6),
        dbc.Col([
            dcc.Graph(id='boxplot-price-city')
        ], width=6)
    ]),
    
    html.Hr(),
    
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='amenities-impact')
        ], width=6),
        dbc.Col([
            dcc.Graph(id='price-segmentation')
        ], width=6)
    ])
])

# Callbacks
@app.callback(
    [Output('scatter-price-sqft', 'figure'),
     Output('boxplot-price-city', 'figure'),
     Output('amenities-impact', 'figure'),
     Output('price-segmentation', 'figure')],
    [Input('city-dropdown', 'value'),
     Input('state-dropdown', 'value')]
)
def update_graphs(selected_cities, selected_states):
    filtered_df = df.copy()
    
    if selected_cities:
        city_cols = [col for col in df.columns if col in selected_cities]
        filtered_df = filtered_df[(filtered_df[city_cols] == 1).any(axis=1)]
    
    if selected_states:
        state_cols = [col for col in df.columns if col in selected_states]
        filtered_df = filtered_df[(filtered_df[state_cols] == 1).any(axis=1)]
    
    scatter_fig = px.scatter(filtered_df, x='square_feet', y='y_actual', color='price_category',
                             title="Precio vs. Metros Cuadrados", labels={'y_actual': 'Precio'})
    
    boxplot_fig = px.box(filtered_df, x='price_category', y='y_actual',
                          title="Distribución de precios por categoría")
    
    amenities_cols = [col for col in df.columns if col in ['Cable or Satellite', 'Pool', 'Parking']]
    amenities_impact = filtered_df[amenities_cols + ['y_actual']].melt(id_vars='y_actual')
    amenities_fig = px.box(amenities_impact, x='variable', y='y_actual', title="Impacto de Amenidades en Precio")
    
    segmentation_fig = px.histogram(filtered_df, x='price_category', title="Segmentación de Precios")
    
    return scatter_fig, boxplot_fig, amenities_fig, segmentation_fig

# Ejecutar el servidor
if __name__ == '__main__':
    app.run_server(debug=True)
